In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data = pd.read_csv('train.csv')
data.shape

(1460, 81)

In [6]:
#getting rid of all the columns that have too many NaN

print(data.isna().sum().sort_values(ascending=False))
data.drop(columns=['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'LotFrontage'], 
          inplace=True)

In [9]:
#List of other variables that have a considerable number of NaN but I want to replace with something
data.isna().sum().sort_values(ascending=False)

con_na = ['GarageType', 
          'GarageYrBlt', 
          'GarageFinish', 
          'GarageCond', 
          'GarageQual', 
          'BsmtExposure', 
          'BsmtFinType2', 
          'BsmtFinType1', 
          'BsmtCond', 
          'BsmtQual']

#Removed GarageYrBlt from the list as it is the only numerical field, I will treat it differently
con_na.remove('GarageYrBlt')

In [14]:
#Fillna for all the variables in the list with 'NA'
data[con_na] = data[con_na].fillna('NA')
#Fillna for GarageYrBlt with 0
data['GarageYrBlt'] = data['GarageYrBlt'].fillna(0)

In [18]:
#Filling other variables that have less NaN
print(data.isna().sum().sort_values(ascending=False))
data['MasVnrType'] = data['MasVnrType'].fillna('None')
data['MasVnrArea'] = data['MasVnrArea'].fillna(0.0)
data['Electrical'] = data['Electrical'].fillna('NA')

In [46]:
#Now we have no values missing
data.isna().sum().sort_values(ascending=False).head()

TotalPorchSF    0
SalePrice       0
RoofStyle       0
RoofMatl        0
Exterior1st     0
dtype: int64

In [37]:
data.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


In [40]:
#Most of the properties have only one type of Porch, and we have 4 columns for the area of each
#Type of porch... So I will create a TotalPorchSF column with the sum of the other 4
#And drop the other for
data[['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']].head(20)

,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch
0,61,0,0,0
1,0,0,0,0
2,42,0,0,0
3,35,272,0,0
4,84,0,0,0
5,30,0,320,0
6,57,0,0,0
7,204,228,0,0
8,0,205,0,0
9,4,0,0,0


In [42]:
data['TotalPorchSF'] = data['OpenPorchSF'] + data['EnclosedPorch'] + data['3SsnPorch'] + data['ScreenPorch']

In [43]:
data.drop(columns=['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'], inplace=True)

In [71]:
#We have 4 columns that give us information about basement area:
#BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF
#The first 2 are the area of finished basement, the 3rd the area of unfinished basement
#And the 4th is the total.
#In order to have less columns, we will compute the % of finished basement area
#((BsmtFinSF1+BsmtFinSF2)/TotalBsmtSF)
#After that, we will eliminate BsmtFinSF1, BsmtFinSF2, BsmtUnfSF,

data['PercentBsmtFin'] = (data.BsmtFinSF1+data.BsmtFinSF2)/data.TotalBsmtSF
data.drop(columns=['BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF'], inplace=True)

37

In [73]:
#we had 37 calculations that could not be made (the ones that have no basement)
#We will replace them with 0
data['PercentBsmtFin'].isna().sum()
data['PercentBsmtFin'] = data['PercentBsmtFin'].fillna(0)

In [77]:
#Now, we have 4 variables that describe the number of bathrooms in each property
#Though it gives us a lot of detail, we could have only one variable for that
data['TotalBath'] = data.BsmtFullBath + data.BsmtHalfBath + data.FullBath + data.HalfBath
data.drop(columns=['BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath'], inplace=True)

In [76]:
#All columns that are categorical but have an implicit scale in it, 
#I will convert in only one column with a numerical scale

#First, let's separate those columns and check what is their scale
data.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'WoodDeckSF', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCon

In [47]:
#Categorical Variables that have an implicit scale
cat_scale = ['ExterQual', 
             'ExterCond', 
             'BsmtQual', 
             'BsmtCond', 
             'BsmtExposure', 
             'BsmtFinType1', 
             'BsmtFinType2', 
             'HeatingQC', 
             'Functional', 
             'GarageFinish', 
             'GarageQual', 
             'GarageCond']
data[cat_scale]

,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,HeatingQC,Functional,GarageFinish,GarageQual,GarageCond
0,Gd,TA,Gd,TA,No,GLQ,Unf,Ex,Typ,RFn,TA,TA
1,TA,TA,Gd,TA,Gd,ALQ,Unf,Ex,Typ,RFn,TA,TA
2,Gd,TA,Gd,TA,Mn,GLQ,Unf,Ex,Typ,RFn,TA,TA
3,TA,TA,TA,Gd,No,ALQ,Unf,Gd,Typ,Unf,TA,TA
4,Gd,TA,Gd,TA,Av,GLQ,Unf,Ex,Typ,RFn,TA,TA
5,TA,TA,Gd,TA,No,GLQ,Unf,Ex,Typ,Unf,TA,TA
6,Gd,TA,Ex,TA,Av,GLQ,Unf,Ex,Typ,RFn,TA,TA
7,TA,TA,Gd,TA,Mn,ALQ,BLQ,Ex,Typ,RFn,TA,TA
8,TA,TA,TA,TA,No,Unf,Unf,Gd,Min1,Unf,Fa,TA
9,TA,TA,TA,TA,No,GLQ,Unf,Ex,Typ,RFn,Gd,TA


In [63]:
cut_1 = {'NA':0, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5} #'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'GarageQual', 'GarageCond'
cut_2 = {'NA':0, 'Unf':1, 'LwQ':2, 'Rec':3, 'BLQ':4, 'ALQ':5, 'GLQ':6} #'BsmtFinType1', 'BsmtFinType2'
cut_3 = {'NA':0, 'No':1, 'Mn':2, 'Av':3, 'Gd':4} #'BsmtExposure'
cut_4 = {'Sal':0, 'Sev':1, 'Maj2':2, 'Maj1':3, 'Mod':4, 'Min2':5, 'Min1':6, 'Typ':7} #'Functional'
cut_5 = {'NA':0, 'Unf':1, 'RFn':2, 'Fin':3} #'GarageFinish'

def labeling(s, dic):
    return dic[s]

In [84]:
col_cut_1 = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'GarageQual', 'GarageCond']
col_cut_2 = ['BsmtFinType1', 'BsmtFinType2']

for el in col_cut_1:
    data[el] = data[el].apply(lambda x: labeling(x, cut_1))

for el in col_cut_2:
    data[el] = data[el].apply(lambda x: labeling(x, cut_2))

data['BsmtExposure'] = data['BsmtExposure'].apply(lambda x: labeling(x, cut_3))
data['Functional'] = data['Functional'].apply(lambda x: labeling(x, cut_4))
data['GarageFinish'] = data['GarageFinish'].apply(lambda x: labeling(x, cut_5))